# How People Decide what they want to do
Directed graph approach

Generally people want to do a number of different things. For this I'm going to create a schema for this in a graph language that allows me to designate how much a `pop` desires to take a certain action. This will be used later when determining AI decisions. 

In [24]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
import nb_black

In [9]:
res = db.run_query("g.V().hasLabel('pop').has('username','userbill').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': '0.197',
 'literacy': '0.822',
 'aggression': '0.384',
 'constitution': '0.615',
 'objid': '5771833830958',
 'faction_no': '2',
 'name': 'Heznius Ying',
 'isInFaction': '7100167726406',
 'industry': '0.4995',
 'wealth': '0.66075',
 'faction_loyalty': '0.49550000000000005',
 'username': 'userbill',
 'objtype': 'pop',
 'id': '5771833830958'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [30]:
n = 2.5
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
# Test equation:
# ind_df = pd.DataFrame(range(10),columns=['wealth'])
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='wealth',y='desires_industry')

alt.Chart(...)